In [1]:
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt

sys.path.insert(0, './icenumerics/')

import icenumerics as ice

ureg = ice.ureg

%reload_ext autoreload
%autoreload 2

idx = pd.IndexSlice

In [2]:
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['axes.grid'] = True

In [3]:
params = {
    "particle_radius":1.4*ureg.um,
    "particle_susceptibility":0.4,
    "particle_diffusion":0.14*ureg.um**2/ureg.s,
    "particle_temperature":300*ureg.K,
    "particle_density":1000*ureg.kg/ureg.m**3,

    "trap_sep":3*ureg.um,
    "trap_height":8*ureg.pN*ureg.nm,
    "trap_stiffness":100e-6*ureg.pN/ureg.nm,
    "height_spread":0,
    "susceptibility_spread":0,
    "isperiodic":True,
    "size" : 10,

    "total_time":120*ureg.s,
    "framespersec":1000*ureg.Hz,
    "dt":1*ureg.ms,
    "max_field":5*ureg.mT,
    "sim_temp":300*ureg.K,
    "sim_dipole_cutoff":40*ureg.um,
}

params["lattice_constant"] = params["trap_sep"]+(2*params["particle_radius"]+1*ureg.um)*np.sqrt(2)

locals().update(params)

https://pint.readthedocs.io/en/0.10.1/serialization.html

In [4]:
particle = ice.particle(radius = particle_radius,
            susceptibility = particle_susceptibility,
            diffusion = particle_diffusion,
            temperature = particle_temperature,
            density = particle_density)

trap = ice.trap(trap_sep = trap_sep,
            height = trap_height,
            stiffness = trap_stiffness) 

sp = ice.spins()
sp.create_lattice("square",[size,size],lattice_constant=lattice_constant, border="periodic")

col = ice.colloidal_ice(sp, particle, trap,
                        height_spread = height_spread, 
                        susceptibility_spread = susceptibility_spread,
                        periodic = isperiodic)


In [5]:
col.randomize()


col.region = np.array(
    [[0,0,-3*(particle_radius/lattice_constant/size).magnitude],
     [1,1,3*(particle_radius/lattice_constant/size).magnitude]])*size*lattice_constant

In [6]:
world = ice.world(
        field = max_field,
        temperature = sim_temp,
        dipole_cutoff = sim_dipole_cutoff,
        boundaries=['p', 'p', 'p'])

col.simulation(world,
            name = "test_0",
            include_timestamp = False,
            targetdir = r".",
            framerate = framespersec,
            timestep = dt,
            run_time = total_time,
            output = ["x","y","z","mux","muy","muz"])

# Field

angle = 0*ureg("deg")
angle = angle.to("rad").magnitude

col.sim.field.fieldx = f"v_Bmag*sin({angle})*time/60e6*(time<60e6)+v_Bmag*sin({angle})*(time>=60e6)"
col.sim.field.fieldy = "0"
col.sim.field.fieldz = f"v_Bmag*cos({angle})*time/60e6*(time<60e6)+v_Bmag*cos({angle})*(time>=60e6)"

In [ ]:
out = col.run_simulation()

col.load_simulation(sl = slice(-100,-1,1))

In [ ]:
col.display()

In [ ]:
world = ice.world(
        field = max_field,
        temperature = sim_temp,
        dipole_cutoff = sim_dipole_cutoff,
        boundaries=['p', 'p', 'p'])

col.simulation(world,
            name = "test_90",
            include_timestamp = False,
            targetdir = r".",
            framerate = framespersec,
            timestep = dt,
            run_time = total_time,
            output = ["x","y","z","mux","muy","muz"])

# Field

angle = 90*ureg("deg")
angle = angle.to("rad").magnitude

col.sim.field.fieldx = f"v_Bmag*sin({angle})*time/60e6*(time<60e6)+v_Bmag*sin({angle})*(time>=60e6)"
col.sim.field.fieldy = "0"
col.sim.field.fieldz = f"v_Bmag*cos({angle})*time/60e6*(time<60e6)+v_Bmag*cos({angle})*(time>=60e6)"

out = col.run_simulation()

col.load_simulation(sl = slice(-100,-1,1))

In [ ]:
out

In [ ]:
col.trj.index.get_level_values("frame").unique()